In [1]:
import typing as tp
import httpx
import json
import re

class LLM:
    def __init__(self, llm_chat_url: str):
        self.url = llm_chat_url
        self.client = httpx.Client(headers={
            "accept": "text/event-stream",
            "accept-language": "en-US,en;q=0.9,ru-RU;q=0.8,ru;q=0.7",
            "content-type": "application/json",
            "cookie": "dcm=5",
            "dnt": "1",
            "origin": "https://duckduckgo.com",
            "priority": "u=1, i",
            "referer": "https://duckduckgo.com/",
            "sec-ch-ua": '"Google Chrome";v="129", "Not=A?Brand";v="8", "Chromium";v="129"',
            "sec-ch-ua-mobile": "?0",
            "sec-ch-ua-platform": '"macOS"',
            "sec-fetch-dest": "empty",
            "sec-fetch-mode": "cors",
            "sec-fetch-site": "same-origin",
            "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36",
            "x-vqd-4": "4-189727535158365797383025471132521611675",
        })


    def _assemble_body(self, prompt: str,
                        model: tp.Literal["claude-3-haiku-20240307",
            "meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo",
            "mistralai/Mixtral-8x7B-Instruct-v0.1",
            "gpt-4o-mini"] = "meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo"):
        return {
            "model": model,
            "messages": [
                {"role": "user", "content": prompt}
            ],
        }

    def __parse_chunk(self, chunk: str) -> str :
        if len(chunk) == 0:
            return ""
        payload = chunk[5:]
        try:
            return json.loads(payload)["message"]
        except Exception as e:
            print(e, chunk)
            return ""


    def _email_prompt(self, vacancy_description: str ) -> str:
        return f"Помоги написать письмо для рассылки с новой вакансией для компании в формате html, вот описание вакансии: {vacancy_description}"


    def __parse_html_response(self, response: str) -> str:
        match = re.search(r'```(.*?)```', response, re.DOTALL)
        if match:
            return match.group(1).strip()
        return ""

    def _telegram_prompt(self, vacancy_description: str) -> str:
        return f"Помоги написать пост для рекламы вакансии в соц сети с использованием markdown, вот описание вакансии: {vacancy_description}"

    def prompt(self, post_type: tp.Literal["telegram", "email"], vacancy_description: str) -> str:
        match post_type:
            case "telegram":
                prompt = self._telegram_prompt(vacancy_description)
            case "email":
                prompt = self._email_prompt(vacancy_description)

        response = ""
        with self.client.stream("POST", self.url,  json=self._assemble_body(prompt), timeout=120) as r:
            for chunk in r.iter_lines():
                new_data = self.__parse_chunk(chunk)
                response += new_data
        match post_type:
            case "telegram":
                response = response
            case "email":
                response = self.__parse_html_response(response)

        return response

In [2]:
DUCK_DUCK_GO_URL = "https://duckduckgo.com/duckchat/v1/chat"
description = """
Мы ищем data scientist’а с опытом работы в текстовых задачах, рекомендательных или поисковых системах. Если вы хорошо знаете математику, любите копаться в больших массивах данных, находить в них закономерности, и умеете внедрять свои решения в продакшн, то мы будем рады видеть вас в нашей компании.

Мы развиваем ML-модели ранжирования поиска и рекомендаций, которые позволяют находить работу людям со всей страны. Наша основная задача состоит в количественном росте продуктовых метрик, таких как кол-во откликов/приглашений, все гипотезы строго проверяются с помощью A/B-экспериментов. Наши решения работают в online режиме под большой нагрузкой на поисковом кластере из 100+ машин. Мы работаем в продуктовых кросс-функциональных (frontend/backend/data science) командах по гибким методологиям.

Требования

Опыт реализации и применения моделей машинного обучения в продакшн;

Понимание основных методов и алгоритмов Machine Learning;

Знание принципов работы нейросетевых моделей для обработки текста;

Опыт в ранжировании, рекомендациях, а также в nlp-задачах будет большим плюсом;

Уверенное владение Python (numpy, scipy, pytorch, sklearn), приветствуется знание и опыт с Java;

Алгоритмическая подготовка: знание основных алгоритмов и структур данных;

Желателен опыт работы с различными источниками данных: SQL, Cassandra, Hive, Presto, Kafka, etc.;

Понимание и опыт работы с A/B-экспериментами;

Обязанности

Развитие моделей поискового ранжирования и рекомендательных систем;

Тюнинг существующих моделей, feature engineering, а также построение новых решений с нуля;

Прототипирование и доведение своих решений до продакшна на python;

Полная поддержка своих моделей: от написания запроса для выгрузки обучающей выборки из хранилищ данных до анализа результатов A/B-экспериментов и переключения всех пользователей на новый функционал;
"""

In [4]:
llm = LLM(DUCK_DUCK_GO_URL)

response = llm.prompt("email", description)
print(response)

Expecting value: line 1 column 3 (char 2) data: [DONE]
<html>
  <head>
    <title>Вакансия Data Scientist в нашей компании</title>
  </head>
  <body>
    <h1>Вакансия Data Scientist в нашей компани��</h1>
    <p>Мы ищем data scientist'а с опытом работы в текстовых задачах, рекомендательных или поисковых системах. Если вы хорошо знаете математику, любите копаться в больших массивах данных, находить в них закономерности, и умеете внедрять свои решения в продакшн, то мы будем рады видеть вас в нашей компании.</p>
    <p>Мы развиваем ML-модели ранжирования поиска и рекомендаций, которые позволяют находить работу людям со всей страны. Наша основная задача состоит в количественном росте продуктовых метрик, таких как кол-во откликов/приглашений, все гипотезы строго проверяются с помощью A/B-экспериментов. Наши решения работают в online режиме под большой нагрузкой на поисковом кластере из 100+ машин. Мы работаем в продуктовых кросс-функциональных (frontend/backend/data science) командах по ги

In [3]:
llm = LLM(DUCK_DUCK_GO_URL)

response = llm.prompt("telegram", description)
print(response)

Expecting value: line 1 column 3 (char 2) data: [DONE]
**Вакансия: Data Scientist в компании по разработке ML-моделей**

Мы ищем опытного Data Scientist'а, который поможет нам улучшить наши ML-модели ранжирования поиска и рекомендаций. Если вы любите работать с большими данными, находить закономерности и внедрять свои решения в продакшн, то мы хотим видеть вас в нашей команде!

**О нас**

Мы разрабатываем ML-модели, которые помогают людям найти работу по всей стране. Наша основная задача - количественный рост продуктовых метрик, таких как количество откликов/приглашений. Мы работаем в продуктовых кросс-функциональных командах по гибким методологиям.

**Требования**

* Опыт реализации и применения моделей машинного обучения в продакшн
* Понимание основных методов и алгоритмов Machine Learning
* Знание принципов работы нейросетевых моделей для обработки текста
* Опыт в ранжировании, рекомендациях, а также в nlp-задачах будет большим плюсом
* Уверенное владение Python (numpy, scipy, pytor